# DICE - Notebook 2 - Dataset Preparation

<br/>

```
*************************************************************************
**
** 2017 Mai 23
**
** In place of a legal notice, here is a blessing:
**
**    May you do good and not evil.
**    May you find forgiveness for yourself and forgive others.
**    May you share freely, never taking more than you give.
**
*************************************************************************
```

<table style="width:100%; font-size:14px; margin: 20px 0;">
    <tr>
        <td style="text-align:center">
            <b>Contact: </b><a href="mailto:contact@jonathandekhtiar.eu" target="_blank">contact@jonathandekhtiar.eu</a>
        </td>
        <td style="text-align:center">
            <b>Twitter: </b><a href="https://twitter.com/born2data" target="_blank">@born2data</a>
        </td>
        <td style="text-align:center">
            <b>Tech. Blog: </b><a href="http://www.born2data.com/" target="_blank">born2data.com</a>
        </td>
    </tr>
    <tr>
        <td style="text-align:center">
            <b>Personal Website: </b><a href="http://www.jonathandekhtiar.eu" target="_blank">jonathandekhtiar.eu</a>
        </td>
        <td style="text-align:center">
            <b>RSS Feed: </b><a href="https://www.feedcrunch.io/@dataradar/" target="_blank">FeedCrunch.io</a>
        </td>
        <td style="text-align:center">
            <b>LinkedIn: </b><a href="https://fr.linkedin.com/in/jonathandekhtiar" target="_blank">JonathanDEKHTIAR</a>
        </td>
    </tr>
</table>

## Objectives

This notebook aims to preprocess and prepare the dataset for later used during the training phase. 

There exists many methods to feed data into a Deep Learning with [Tensorflow](https://www.tensorflow.org/), the Python Library we have chosen to use for this study:

1. **From Disk**: Data can be inputed into a model with the **feed_dict** argument when running a *training operation*. It would  definitely be possible, however this process can be slow if there are a lot of data to read simultaneously and could be too large to be held in the GPU Memory.
<br><br>
2. **From a CSV File**: This [type of file](https://en.wikipedia.org/wiki/Comma-separated_values) is not revelant when dealing with images.
<br><br>
3. **From a preprocessed binary file**: Tensorflow is able to save and recover data in a binary format called [TFRecords](https://www.tensorflow.org/api_guides/python/python_io#TFRecords_Format_Details). The data can be preprocessed beforehand and only the necessary data can be saved and read in real time during the training. This approach is the fatest and most memory-efficient when dealing with images.

This notebook will focus on generating the necessary **TFRecord** files. Generating **TFRecords** is less intuitive than 
[HDF5](https://en.wikipedia.org/wiki/Hierarchical_Data_Format), used in other Deep Learning libraries such as [Keras](https://keras.io/). Using **TFRecords** will give you access to natively available tools, such as *Queue Runners*, *Coordinators*, *Supervisors*, *etc.*, to design [data pipelines](https://www.tensorflow.org/programmers_guide/reading_data) and process the images in a batch fashion.

This notebook will use [Tensorflow-Slim](https://github.com/tensorflow/tensorflow/tree/master/tensorflow/contrib/slim) to ease the understanding and reduce the code complexity.

As we aim to to later re-train a CNN Model

This will be used later to retrain an CNN model: [Inception-V4](https://arxiv.org/abs/1602.07261) model developed by Szegedy et al. The model has been Pre-Trained with the [ImageNet](http://www.image-net.org/) dataset allowing a much more accurate result due to the large number of data avaiable in this dataset. We call this kind of process: "*Transfer Learning*".


This notebook will also randomly split the available data into two sets of data: [Training and Validation sets](https://stats.stackexchange.com/questions/19048/what-is-the-difference-between-test-set-and-validation-set). This process aims to reduce the [overfit](http://scikit-learn.org/stable/auto_examples/model_selection/plot_underfitting_overfitting.html) of the model and thus improving its accuracy on previously unseen data. 

In this study the selection ratio has been chosen as followed:
- *training set:* 60%
- *validation set:* 40%.

---

As reminder before starting, the data have already been preprocessed (resized, augmented, etc.) in the first Notebook: **[DICE - Notebook 1 - Dataset Augmentation](https://github.com/DEKHTIARJonathan/DICE-DMU_Imagery_Classification_Engine/blob/master/DICE%20-%20Notebook%201%20-%20Dataset%20Augmentation.ipynb)**

The preprocessed data all have been saved as **JPEG images** and thus we will only focus on these data.

## 1. Notebook Initialisation

### 1.1. Load the necessary libraries

In [1]:
import os, time, math

import tensorflow as tf
from tensorflow.contrib.framework.python.ops.variables import get_or_create_global_step
from tensorflow.python.platform import tf_logging as logging

import inception_preprocessing
from inception_resnet_v2 import inception_resnet_v2, inception_resnet_v2_arg_scope

slim = tf.contrib.slim

In [2]:
#================ DATASET INFORMATION ======================
#State dataset directory where the tfrecord files are located
dataset_dir = 'data_prepared/'

#State where your log file is at. If it doesn't exist, create it.
log_dir = 'log/'

#Create the log directory here. Must be done here otherwise import will activate this unneededly.
if not os.path.exists(log_dir):
    os.mkdir(log_dir)

#State where your checkpoint file is
checkpoint_file = 'model/inception_resnet_v2_2016_08_30.ckpt'

#State the image size you're resizing your images to. We will use the default inception size of 299.
image_size = 299

#State the number of classes to predict:
num_classes = 5

#State the labels file and read it
labels_file = 'data_prepared/labels.txt'
labels = open(labels_file, 'r')

#Create a dictionary to refer each label to their string name
labels_to_name = {}
for line in labels:
    label, string_name = line.split(':')
    string_name = string_name[:-1] #Remove newline
    labels_to_name[int(label)] = string_name

#Create the file pattern of your TFRecord files so that it could be recognized later on
tf_record_start_name = 'dmunet_dataset_'
file_pattern = tf_record_start_name + '%s_*.tfrecord'

#Create a dictionary that will help people understand your dataset better. This is required by the Dataset class later.
items_to_descriptions = {
    'image': 'A 3-channel RGB coloured flower image that is either tulips, sunflowers, roses, dandelion, or daisy.',
    'label': 'A label that is as such -- 0:daisy, 1:dandelion, 2:roses, 3:sunflowers, 4:tulips'
}

tf.logging.set_verbosity(tf.logging.INFO) #Set the verbosity to INFO level

#================= TRAINING INFORMATION ==================
#State the number of epochs to train
num_epochs = 30

#State your batch size => Choose the highest value which doesn't give you a memory error.
batch_size = 14

#Learning rate information and configuration (Up to you to experiment)
initial_learning_rate = 0.0002
learning_rate_decay_factor = 0.7
num_epochs_before_decay = 2

In [3]:
#============== DATASET LOADING ======================
#We now create a function that creates a Dataset class which will give us many TFRecord files to feed in the examples into a queue in parallel.
def get_split(split_name, dataset_dir, file_pattern=file_pattern):
    '''
    Obtains the split - training or validation - to create a Dataset class for feeding the examples into a queue later on. This function will
    set up the decoder and dataset information all into one Dataset class so that you can avoid the brute work later on.
    Your file_pattern is very important in locating the files later. 

    INPUTS:
    - split_name(str): 'train' or 'validation'. Used to get the correct data split of tfrecord files
    - dataset_dir(str): the dataset directory where the tfrecord files are located
    - file_pattern(str): the file name structure of the tfrecord files in order to get the correct data

    OUTPUTS:
    - dataset (Dataset): A Dataset class object where we can read its various components for easier batch creation later.
    '''

    #First check whether the split_name is train or validation
    if split_name not in ['train', 'validation']:
        raise ValueError('The split_name %s is not recognized. Please input either train or validation as the split_name' % (split_name))

    #Create the full path for a general file_pattern to locate the tfrecord_files
    file_pattern_path = os.path.join(dataset_dir, file_pattern % (split_name))

    #Count the total number of examples in all of these shard
    num_samples = 0
    file_pattern_for_counting = tf_record_start_name + split_name
    tfrecords_to_count = [os.path.join(dataset_dir, file) for file in os.listdir(dataset_dir) if file.startswith(file_pattern_for_counting)]
    for tfrecord_file in tfrecords_to_count:
        for record in tf.python_io.tf_record_iterator(tfrecord_file):
            num_samples += 1

    #Create a reader, which must be a TFRecord reader in this case
    reader = tf.TFRecordReader

    #Create the keys_to_features dictionary for the decoder
    keys_to_features = {
      'image/encoded': tf.FixedLenFeature((), tf.string, default_value=''),
      'image/format': tf.FixedLenFeature((), tf.string, default_value='jpg'),
      'image/class/label': tf.FixedLenFeature(
          [], tf.int64, default_value=tf.zeros([], dtype=tf.int64)),
    }

    #Create the items_to_handlers dictionary for the decoder.
    items_to_handlers = {
    'image': slim.tfexample_decoder.Image(),
    'label': slim.tfexample_decoder.Tensor('image/class/label'),
    }

    #Start to create the decoder
    decoder = slim.tfexample_decoder.TFExampleDecoder(keys_to_features, items_to_handlers)

    #Create the labels_to_name file
    labels_to_name_dict = labels_to_name

    #Actually create the dataset
    dataset = slim.dataset.Dataset(
        data_sources = file_pattern_path,
        decoder = decoder,
        reader = reader,
        num_readers = 4,
        num_samples = num_samples,
        num_classes = num_classes,
        labels_to_name = labels_to_name_dict,
        items_to_descriptions = items_to_descriptions)

    return dataset

In [4]:
def load_batch(dataset, batch_size, height=image_size, width=image_size, is_training=True):
    '''
    Loads a batch for training.

    INPUTS:
    - dataset(Dataset): a Dataset class object that is created from the get_split function
    - batch_size(int): determines how big of a batch to train
    - height(int): the height of the image to resize to during preprocessing
    - width(int): the width of the image to resize to during preprocessing
    - is_training(bool): to determine whether to perform a training or evaluation preprocessing

    OUTPUTS:
    - images(Tensor): a Tensor of the shape (batch_size, height, width, channels) that contain one batch of images
    - labels(Tensor): the batch's labels with the shape (batch_size,) (requires one_hot_encoding).

    '''
    #First create the data_provider object
    data_provider = slim.dataset_data_provider.DatasetDataProvider(
        dataset,
        common_queue_capacity = 24 + 3 * batch_size,
        common_queue_min = 24)

    #Obtain the raw image using the get method
    raw_image, label = data_provider.get(['image', 'label'])

    #Perform the correct preprocessing for this image depending if it is training or evaluating
    image = inception_preprocessing.preprocess_image(raw_image, height, width, is_training)

    #As for the raw images, we just do a simple reshape to batch it up
    raw_image = tf.expand_dims(raw_image, 0)
    raw_image = tf.image.resize_nearest_neighbor(raw_image, [height, width])
    raw_image = tf.squeeze(raw_image)

    #Batch up the image by enqueing the tensors internally in a FIFO queue and dequeueing many elements with tf.train.batch.
    images, raw_images, labels = tf.train.batch(
        [image, raw_image, label],
        batch_size = batch_size,
        num_threads = 4,
        capacity = 4 * batch_size,
        allow_smaller_final_batch = True)

    return images, raw_images, labels

### Loading dataset and data batches

In [5]:
dataset = get_split('train', dataset_dir, file_pattern=file_pattern)
images, _, labels = load_batch(dataset, batch_size=batch_size)

num_batches_per_epoch = tf.cast(tf.ceil(dataset.num_samples / batch_size), tf.int32)

#Know the number steps to take before decaying the learning rate and batches per epoch
num_steps_per_epoch = num_batches_per_epoch #Because one step is one batch processed
decay_steps = tf.cast(num_epochs_before_decay * num_steps_per_epoch, tf.int32)

In [6]:
#Create the model inference
with slim.arg_scope(inception_resnet_v2_arg_scope()):
    logits, end_points = inception_resnet_v2(images, num_classes = dataset.num_classes, is_training = True)

In [7]:
#Define the scopes that you want to exclude for restoration
exclude = ['InceptionResnetV2/Logits', 'InceptionResnetV2/AuxLogits']
variables_to_restore = slim.get_variables_to_restore(exclude = exclude)

In [8]:
#Perform one-hot-encoding of the labels (Try one-hot-encoding within the load_batch function!)
one_hot_labels = slim.one_hot_encoding(labels, dataset.num_classes)

In [9]:
#Performs the equivalent to tf.nn.sparse_softmax_cross_entropy_with_logits but enhanced with checks
loss = tf.losses.softmax_cross_entropy(onehot_labels = one_hot_labels, logits = logits)
total_loss = tf.losses.get_total_loss()    #obtain the regularization losses as well

In [10]:
#Create the global step for monitoring the learning_rate and training.
global_step = get_or_create_global_step()

In [11]:
#Define your exponentially decaying learning rate
lr = tf.train.exponential_decay(
    learning_rate = initial_learning_rate,
    global_step = global_step,
    decay_steps = decay_steps,
    decay_rate = learning_rate_decay_factor,
    staircase = True
)

In [12]:
#Now we can define the optimizer that takes on the learning rate
optimizer = tf.train.AdamOptimizer(learning_rate = lr)

In [13]:
#Create the train_op.
train_op = slim.learning.create_train_op(total_loss, optimizer)

In [14]:
#State the metrics that you want to predict. We get a predictions that is not one_hot_encoded.
predictions = tf.argmax(end_points['Predictions'], 1)
probabilities = end_points['Predictions']
accuracy, accuracy_update = tf.contrib.metrics.streaming_accuracy(predictions, labels)
metrics_op = tf.group(accuracy_update, probabilities)

In [15]:
#Now finally create all the summaries you need to monitor and group them into one summary op.
tf.summary.scalar('losses/Total_Loss', total_loss)
tf.summary.scalar('accuracy', accuracy)
tf.summary.scalar('learning_rate', lr)
my_summary_op = tf.summary.merge_all()

In [16]:
#Now we need to create a training step function that runs both the train_op, metrics_op and updates the global_step concurrently.
def train_step(sess, train_op, global_step):
    '''
    Simply runs a session for the three arguments provided and gives a logging on the time elapsed for each global step
    '''
    #Check the time for each sess run
    start_time = time.time()
    total_loss, global_step_count, _ = sess.run([train_op, global_step, metrics_op])
    time_elapsed = time.time() - start_time

    #Run the logging to print some results
    logging.info('global step %s: loss: %.4f (%.2f sec/step)', global_step_count, total_loss, time_elapsed)

    return total_loss, global_step_count

In [17]:
#Now we create a saver function that actually restores the variables from a checkpoint file in a sess
saver = tf.train.Saver(variables_to_restore)
def restore_fn(sess):
    return saver.restore(sess, checkpoint_file)

In [ ]:
#Define your supervisor for running a managed session. 
#Do not run the summary_op automatically or else it will consume too much memory

sv = tf.train.Supervisor(logdir = log_dir, summary_op = None, init_fn = restore_fn)

In [ ]:
#Run the managed session
with sv.managed_session() as sess:
    
    _num_steps_per_epoch, _num_batches_per_epoch = sess.run([num_steps_per_epoch, num_batches_per_epoch])    
    
    print("num_steps_per_epoch:", _num_steps_per_epoch)  
    print("num_batches_per_epoch:", _num_batches_per_epoch)
    print("num_epochs:", num_epochs)
    
    print("total:", _num_steps_per_epoch * num_epochs)
    
    for step in range(_num_steps_per_epoch * num_epochs):
        
        #At the start of every epoch, show the vital information:
        if step % _num_batches_per_epoch == 0:
            logging.info('Epoch %s/%s', step/_num_batches_per_epoch + 1, num_epochs)
            learning_rate_value, accuracy_value = sess.run([lr, accuracy])
            logging.info('Current Learning Rate: %s', learning_rate_value)
            logging.info('Current Streaming Accuracy: %s', accuracy_value)

            # optionally, print your logits and predictions for a sanity check that things are going fine.
            logits_value, probabilities_value, predictions_value, labels_value = sess.run([
                logits, 
                probabilities, 
                predictions, 
                labels
            ])
            
            print ('logits: \n', logits_value)
            print ('Probabilities: \n', probabilities_value)
            print ('predictions: \n', predictions_value)
            print ('Labels:\n:', labels_value)

        #Log the summaries every 10 step.
        if step % 10 == 0:
            loss, _ = train_step(sess, train_op, sv.global_step)
            summaries = sess.run(my_summary_op)
            sv.summary_computed(sess, summaries)

        #If not, simply run the training step
        else:
            loss, _ = train_step(sess, train_op, sv.global_step)

    #We log the final training loss and accuracy
    logging.info('Final Loss: %s', loss)
    logging.info('Final Accuracy: %s', sess.run(accuracy))

    #Once all the training has been done, save the log files and checkpoint model
    logging.info('Finished training! Saving model to disk now.')
    # saver.save(sess, "./flowers_model.ckpt")
    sv.saver.save(sess, sv.save_path, global_step = sv.global_step)

INFO:tensorflow:Restoring parameters from log/model.ckpt-0
INFO:tensorflow:global_step/sec: 0
num_steps_per_epoch: 4876
num_batches_per_epoch: 4876
num_epochs: 30
total: 146280
INFO:tensorflow:Epoch 1.0/30
INFO:tensorflow:Current Learning Rate: 0.0002
INFO:tensorflow:Current Streaming Accuracy: 0.0
logits: 
 [[-0.77252555  0.4478153  -0.018868    0.16657667 -0.05937041]
 [-0.6278286  -0.29029846 -0.20948823 -0.24767564 -0.27595681]
 [-0.11307829 -0.04914204 -0.67847419 -0.54170096 -0.48865983]
 [-0.86079305 -0.49302948  0.4180305   0.58822399 -0.48470998]
 [-0.79382354  0.2182719  -0.12678927  0.59697485  0.06259594]
 [-0.39356339 -0.67539716  0.37656036  0.3637068  -1.06576049]
 [-0.53375268  0.02022753 -0.2114595  -0.01591312 -0.71770221]
 [-0.3647446  -0.70912516  0.18293066  0.75264382 -0.15489998]
 [-0.49692446  0.32737732 -0.06713459 -0.08677792 -0.32306647]
 [-0.29924527  0.34148335 -0.28721502 -0.081971   -0.45514295]
 [-0.31555846 -0.192754   -0.51218784  0.59839284 -0.9803270

INFO:tensorflow:global step 100: loss: 1.0641 (1.33 sec/step)
INFO:tensorflow:global step 101: loss: 0.8401 (1.33 sec/step)
INFO:tensorflow:global step 102: loss: 0.9995 (1.33 sec/step)
INFO:tensorflow:global step 103: loss: 0.7136 (1.33 sec/step)
INFO:tensorflow:global step 104: loss: 0.7198 (1.33 sec/step)
INFO:tensorflow:global step 105: loss: 0.9946 (1.33 sec/step)
INFO:tensorflow:global step 106: loss: 1.4333 (1.33 sec/step)
INFO:tensorflow:global step 107: loss: 1.2442 (1.33 sec/step)
INFO:tensorflow:global step 108: loss: 1.1548 (1.33 sec/step)
INFO:tensorflow:global step 109: loss: 1.3021 (1.32 sec/step)
INFO:tensorflow:global step 110: loss: 1.5445 (1.33 sec/step)
INFO:tensorflow:global step 111: loss: 1.1017 (1.32 sec/step)
INFO:tensorflow:global step 112: loss: 1.0193 (1.34 sec/step)
INFO:tensorflow:global step 113: loss: 0.8750 (1.33 sec/step)
INFO:tensorflow:global step 114: loss: 0.8778 (1.33 sec/step)
INFO:tensorflow:global step 115: loss: 0.9003 (1.32 sec/step)
INFO:ten

INFO:tensorflow:global step 232: loss: 1.6604 (1.36 sec/step)
INFO:tensorflow:global step 233: loss: 1.0595 (1.36 sec/step)
INFO:tensorflow:global step 234: loss: 0.6796 (1.36 sec/step)
INFO:tensorflow:global step 235: loss: 0.7720 (1.36 sec/step)
INFO:tensorflow:global step 236: loss: 0.9160 (1.57 sec/step)
INFO:tensorflow:global step 237: loss: 0.9261 (1.34 sec/step)
INFO:tensorflow:global step 238: loss: 0.8252 (1.34 sec/step)
INFO:tensorflow:global step 239: loss: 1.0937 (1.34 sec/step)
INFO:tensorflow:global step 240: loss: 1.0177 (1.35 sec/step)
INFO:tensorflow:global step 241: loss: 1.2502 (1.34 sec/step)
INFO:tensorflow:global step 242: loss: 1.1086 (1.33 sec/step)
INFO:tensorflow:global step 243: loss: 0.9616 (1.33 sec/step)
INFO:tensorflow:global step 244: loss: 1.0058 (1.34 sec/step)
INFO:tensorflow:global step 245: loss: 1.0788 (1.34 sec/step)
INFO:tensorflow:global step 246: loss: 1.0791 (1.33 sec/step)
INFO:tensorflow:global step 247: loss: 1.0186 (1.33 sec/step)
INFO:ten

INFO:tensorflow:global step 363: loss: 1.0522 (1.38 sec/step)
INFO:tensorflow:global step 364: loss: 0.9883 (1.37 sec/step)
INFO:tensorflow:global step 365: loss: 1.0429 (1.38 sec/step)
INFO:tensorflow:global step 366: loss: 0.8861 (1.38 sec/step)
INFO:tensorflow:global step 367: loss: 0.7352 (1.38 sec/step)
INFO:tensorflow:global step 368: loss: 0.6743 (1.38 sec/step)
INFO:tensorflow:global step 369: loss: 0.8630 (1.38 sec/step)
INFO:tensorflow:global step 370: loss: 0.8383 (1.38 sec/step)
INFO:tensorflow:global step 371: loss: 0.7494 (1.38 sec/step)
INFO:tensorflow:global step 372: loss: 0.9027 (1.38 sec/step)
INFO:tensorflow:global step 373: loss: 0.8011 (1.38 sec/step)
INFO:tensorflow:global step 374: loss: 0.6761 (1.38 sec/step)
INFO:tensorflow:global step 375: loss: 1.1660 (1.37 sec/step)
INFO:tensorflow:global step 376: loss: 0.6144 (1.38 sec/step)
INFO:tensorflow:global step 377: loss: 1.2856 (1.37 sec/step)
INFO:tensorflow:global step 378: loss: 1.1093 (1.37 sec/step)
INFO:ten

INFO:tensorflow:global step 495: loss: 1.1795 (1.38 sec/step)
INFO:tensorflow:global step 496: loss: 0.7247 (1.37 sec/step)
INFO:tensorflow:global step 497: loss: 0.7968 (1.38 sec/step)
INFO:tensorflow:global step 498: loss: 1.2630 (1.38 sec/step)
INFO:tensorflow:global step 499: loss: 0.6772 (1.38 sec/step)
INFO:tensorflow:global step 500: loss: 0.9011 (1.38 sec/step)
INFO:tensorflow:global step 501: loss: 0.7192 (1.38 sec/step)
INFO:tensorflow:global_step/sec: 0.674932
INFO:tensorflow:global step 502: loss: 0.7418 (1.37 sec/step)
INFO:tensorflow:global step 503: loss: 0.6865 (1.38 sec/step)
INFO:tensorflow:global step 504: loss: 0.7990 (1.38 sec/step)
INFO:tensorflow:global step 505: loss: 0.7066 (1.38 sec/step)
INFO:tensorflow:global step 506: loss: 1.4242 (1.38 sec/step)
INFO:tensorflow:global step 507: loss: 0.9489 (1.38 sec/step)
INFO:tensorflow:global step 508: loss: 0.9178 (1.38 sec/step)
INFO:tensorflow:global step 509: loss: 0.9233 (1.38 sec/step)
INFO:tensorflow:global step 

...

INFO:tensorflow:global step 1939: loss: 0.7847 (1.38 sec/step)
INFO:tensorflow:global step 1940: loss: 0.5959 (1.37 sec/step)
INFO:tensorflow:global step 1941: loss: 0.6407 (1.37 sec/step)
INFO:tensorflow:global step 1942: loss: 0.5572 (1.38 sec/step)
INFO:tensorflow:global step 1943: loss: 0.9173 (1.37 sec/step)
INFO:tensorflow:global step 1944: loss: 0.5902 (1.38 sec/step)
INFO:tensorflow:global step 1945: loss: 0.5492 (1.38 sec/step)
INFO:tensorflow:global step 1946: loss: 1.2901 (1.37 sec/step)
INFO:tensorflow:global step 1947: loss: 0.7725 (1.38 sec/step)
INFO:tensorflow:global step 1948: loss: 0.7784 (1.38 sec/step)
INFO:tensorflow:global step 1949: loss: 0.7840 (1.37 sec/step)
INFO:tensorflow:global step 1950: loss: 0.8631 (1.38 sec/step)
INFO:tensorflow:global step 1951: loss: 0.8487 (1.37 sec/step)
INFO:tensorflow:global step 1952: loss: 0.7062 (1.37 sec/step)
INFO:tensorflow:global step 1953: loss: 0.7275 (1.38 sec/step)
INFO:tensorflow:global step 1954: loss: 0.6066 (1.38 se

INFO:tensorflow:global step 2070: loss: 0.8184 (1.39 sec/step)
INFO:tensorflow:global step 2071: loss: 0.9684 (1.39 sec/step)
INFO:tensorflow:global step 2072: loss: 1.0939 (1.39 sec/step)
INFO:tensorflow:global step 2073: loss: 0.5908 (1.39 sec/step)
INFO:tensorflow:global step 2074: loss: 0.6103 (1.39 sec/step)
INFO:tensorflow:global step 2075: loss: 0.6071 (1.39 sec/step)
INFO:tensorflow:global step 2076: loss: 1.2350 (1.39 sec/step)
INFO:tensorflow:global step 2077: loss: 1.0268 (1.39 sec/step)
INFO:tensorflow:global step 2078: loss: 0.9653 (1.39 sec/step)
INFO:tensorflow:global step 2079: loss: 0.7143 (1.38 sec/step)
INFO:tensorflow:global step 2080: loss: 0.7211 (1.43 sec/step)
INFO:tensorflow:global step 2081: loss: 0.7371 (1.39 sec/step)
INFO:tensorflow:global step 2082: loss: 1.4429 (1.40 sec/step)
INFO:tensorflow:global step 2083: loss: 0.6744 (1.37 sec/step)
INFO:tensorflow:global step 2084: loss: 0.7394 (1.37 sec/step)
INFO:tensorflow:global step 2085: loss: 0.8038 (1.37 se

INFO:tensorflow:global step 2201: loss: 0.7038 (1.37 sec/step)
INFO:tensorflow:global step 2202: loss: 0.8198 (1.38 sec/step)
INFO:tensorflow:global step 2203: loss: 0.6070 (1.38 sec/step)
INFO:tensorflow:global step 2204: loss: 1.5537 (1.38 sec/step)
INFO:tensorflow:global step 2205: loss: 0.5151 (1.38 sec/step)
INFO:tensorflow:global step 2206: loss: 0.6842 (1.37 sec/step)
INFO:tensorflow:global step 2207: loss: 0.6774 (1.38 sec/step)
INFO:tensorflow:global step 2208: loss: 0.5952 (1.37 sec/step)
INFO:tensorflow:global step 2209: loss: 0.7063 (1.37 sec/step)
INFO:tensorflow:global step 2210: loss: 0.9099 (1.38 sec/step)
INFO:tensorflow:global step 2211: loss: 1.0015 (1.38 sec/step)
INFO:tensorflow:global step 2212: loss: 1.1236 (1.38 sec/step)
INFO:tensorflow:global step 2213: loss: 0.7654 (1.38 sec/step)
INFO:tensorflow:global step 2214: loss: 0.7005 (1.38 sec/step)
INFO:tensorflow:global step 2215: loss: 0.8263 (1.37 sec/step)
INFO:tensorflow:global step 2216: loss: 0.6035 (1.38 se

INFO:tensorflow:global step 2332: loss: 0.6754 (1.40 sec/step)
INFO:tensorflow:global step 2333: loss: 0.8142 (1.39 sec/step)
INFO:tensorflow:global step 2334: loss: 0.8341 (1.39 sec/step)
INFO:tensorflow:global step 2335: loss: 0.7209 (1.39 sec/step)
INFO:tensorflow:global step 2336: loss: 1.1843 (1.39 sec/step)
INFO:tensorflow:global step 2337: loss: 0.5738 (1.39 sec/step)
INFO:tensorflow:global step 2338: loss: 0.7957 (1.39 sec/step)
INFO:tensorflow:global step 2339: loss: 0.7776 (1.76 sec/step)
INFO:tensorflow:global step 2340: loss: 0.8047 (1.34 sec/step)
INFO:tensorflow:global step 2341: loss: 0.6880 (1.35 sec/step)
INFO:tensorflow:global step 2342: loss: 0.8246 (1.36 sec/step)
INFO:tensorflow:global step 2343: loss: 0.7722 (1.38 sec/step)
INFO:tensorflow:global step 2344: loss: 0.6718 (1.44 sec/step)
INFO:tensorflow:global step 2345: loss: 0.5471 (1.43 sec/step)
INFO:tensorflow:global step 2346: loss: 0.7169 (1.40 sec/step)
INFO:tensorflow:global step 2347: loss: 0.6414 (1.42 se

INFO:tensorflow:global step 2463: loss: 0.8849 (1.43 sec/step)
INFO:tensorflow:global step 2464: loss: 0.7834 (1.40 sec/step)
INFO:tensorflow:global step 2465: loss: 0.8125 (1.39 sec/step)
INFO:tensorflow:global step 2466: loss: 0.6052 (1.40 sec/step)
INFO:tensorflow:global step 2467: loss: 0.5684 (1.44 sec/step)
INFO:tensorflow:global step 2468: loss: 0.7123 (1.50 sec/step)
INFO:tensorflow:global step 2469: loss: 0.7038 (1.39 sec/step)
INFO:tensorflow:global step 2470: loss: 1.0123 (1.41 sec/step)
INFO:tensorflow:global step 2471: loss: 0.6239 (1.46 sec/step)
INFO:tensorflow:global step 2472: loss: 0.6329 (1.39 sec/step)
INFO:tensorflow:global step 2473: loss: 0.9800 (1.42 sec/step)
INFO:tensorflow:global step 2474: loss: 0.7240 (1.42 sec/step)
INFO:tensorflow:global step 2475: loss: 1.2526 (1.44 sec/step)
INFO:tensorflow:global step 2476: loss: 0.8532 (1.45 sec/step)
INFO:tensorflow:global step 2477: loss: 0.5581 (1.40 sec/step)
INFO:tensorflow:global step 2478: loss: 0.6513 (1.43 se